In [1]:
import draw

import numpy as np
from collections import defaultdict, Counter
from faker import Faker
from tabulate import tabulate
fake = Faker()

In [23]:
import importlib
importlib.reload(draw)

<module 'draw' from '/Users/cpj/Documents/code/lss/draw/draw.py'>

## Draw Simulation Methods

In [3]:
def init_players(num_players):
    """
    Create a collection of players in the form [player_id, player_name, player_ranking]
    Arguments:
        num_players: Number of players to add to the draw
    Returns:
        players: Array of player details
            (n x 3) list containing [player_id, player_name, player_ranking]
    """
    players = []
    id_to_name, id_to_rank = defaultdict(lambda: 'Bye'), defaultdict(lambda: 1600)
    for n in range(num_players):
        p_id, p_name, p_rank = np.random.randint(1000000, 9999999), fake.name(), np.random.randint(1600, 2000)
        players.append(['a' + str(p_id), p_name, p_rank])
        id_to_name['a' + str(p_id)] = p_name
        id_to_rank['a' + str(p_id)] = p_rank
    return players, id_to_name, id_to_rank

In [4]:
def init_pairings(players):
    """
    Randomly create results for the first round
    Arguments:
        players: (n x 3) numpy array containing [player_id, player_name, player_ranking]
    Returns:
        results: Randomly generated results table
            (n x 3) list containing [player_a_id, player_b_id, match_winner_id]
    """
    results = []
    for n in range(0, len(players), 2):
        a, b = players[n][0], players[n+1][0]
        result = [a, b, np.random.choice([a, b, None], 1, p=(0.45, 0.45, 0.1))[0]]
        results.append(result)
    return results

In [5]:
def run_round(pairings, rnd, p_draw=0):
    """
    Randomly create results for a round
    Arguments:
        pairings: List of pairings for the round to be simulated
            (n x 2 x 2) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id)]
    Returns:
        results: Randomly generated results table
            (n x 3) list containing [player_a_id, player_b_id, match_winner_id]
    """
    results = []
    p_win = (1 - p_draw) / 2
    BYE = 0
    for pairing in pairings:
        a, b, pod = pairing[0][1], pairing[1][1], pairing[2]
        if a[:3] == 'BYE':
            result = [a,b,b]
        elif b[:3] == 'BYE':
            result = [a,b,a]
        else:
            result = [a, b, np.random.choice([a, b, None], 1, p=(p_win, p_win, p_draw))[0]]
        results.append(result + [rnd, pod])
        
    return results

## Draw Printing Methods

In [6]:
def print_standings(standings, id_to_name):
    """
    Print standings for the round in table form
    """
    table = []
    for rank, player in enumerate(standings):
        table.append([player[0], id_to_name[player[0]], player[1], rank+1, player[2], player[5], player[6], player[7], player[8], player[3], player[4]])
    print('\n', tabulate(table, headers=['ID', 'Name', 'Wins', 'Standing', 'CTB', 'PML', 'OML', 'OCTB', 'RPLD', 'Pod', 'Pod Wins']), '\n')
    

def print_pairings(pairs, id_to_name, scores):
    """
    Print pairings for the round in table form
    """
    table = []
    for table_num, pair in enumerate(pairs):
        a, b, pod = pair[0], pair[1], pair[2]
        score = scores[a[0], b[0]]
        row = [table_num+1, pod, id_to_name[a[1]], a[1], int(a[0])+1, id_to_name[b[1]], b[1], int(b[0])+1, score]
        table.append(row)
    headers = ['Table', 'Pod', 'Player 1 - Name', 'ID', 'Rank', 'Player 2 - Name', 'ID', 'Rank', 'GOF']
    print('\n', tabulate(table, headers=headers), '\n')

def print_byes(pairs_h):
    byes = []
    for pairs in pairs_h:
        for pair in pairs:
            a, b = pair[0][1], pair[1][1]
            if a[:3] == 'BYE': byes += [[b], ['BYE']]
            if b[:3] == 'BYE': byes += [[a], ['BYE']]

    byes = Counter(np.reshape(byes, -1))

    if len(byes) > 0:
        print('----- BYES -----')
        for k, v in byes.items():
            if k[:3] == 'BYE':
                print('Byes', ': ', v)
            else:
                print(k, ': ', v)

## Run a sample Tournament
Create a list of players then run through a multi-round tournament and print the results
Each round prints:
- Goodness of fit score: Score for the pairings allocated in the round. The maximum possible score is 4. If a number < 0 comes out, something has probably gone wrong
- Standings: A table of standings of the players (based only on # of wins, no accounting for draws, op wins etc)
- Pairings: A table of pairings for the following round (including player name and standing so you can check if players are being paired with others nearby)

To show all the standings and pairings at once, make sure to select the cell, click on the "Cell" menu, select -> "Current Outputs" and then select "Toggle Scrolling"

The entire history of "standings", "splits", "scores", "allocations", and "pairs" is recorded in the lists *standings_h, splits_h, scores_h, alloc_h, pairs_h*
    
Note: Early stage demo only. This does not support:
- Odd numbers of players and byes [Done]
- Allocating pods [Done]
- Iterative sampling methods to improve draw quality [Done]
- Balancing tail end pod sizes [Partly Done]
- Variable size pods [Done]
- etc

In [7]:
NUMBER_OF_PLAYERS = 31
ROUNDS  = ['swiss1', 'swiss1', 'swiss1', 'swiss1']
NUMBER_OF_ROUNDS = len(ROUNDS)
PROBABILITY_OF_DRAW = 0.1
NUMBER_OF_DROPS = 0

In [8]:
# Create players
players, id_to_name, id_to_rank = init_players(NUMBER_OF_PLAYERS)

In [9]:
importlib.reload(draw)
results = []
standings_h, splits_h, scores_h, alloc_h, pairs_h = [], [], [], [], []

for rnd in range(NUMBER_OF_ROUNDS):
    isLastRound = rnd==len(ROUNDS)-1
    standings, splits, scores, alloc, pairs = draw.create_draw(players, results, ROUNDS,
                                                               mode='test', minimiseTiebreakDist=isLastRound)
    for collection, value in zip([standings_h, splits_h, scores_h, alloc_h, pairs_h],
                                 [standings,   splits,   scores,   alloc,   pairs]):
        collection.append(value)
    n_players = len(standings)
    brackets = [n for n in range(n_players) if (standings[n,[1]] != standings[n-1,[1]]).any()] + [n_players]
    n_odd_brackets = sum(np.array(brackets) % 2)
    max_score = draw.Points.valid - 2 * n_odd_brackets * \
                (draw.Points.valid + draw.Points.overall_win_distance) / n_players
    point_freq = Counter([score for score in scores[alloc==1].astype(int)])

    print('----- Round', rnd + 1, '-----', )
    print('Goodness of Fit:', (scores * alloc).sum() / len(scores))
    print('Maximum GOF:', max_score)
    print('Frequencies: ' + ', '.join([str(int(k)) + 'pts: ' + str(int(point_freq[k] / 2)) 
                                 for k in sorted(point_freq.keys())[::-1]]))
    if NUMBER_OF_PLAYERS <= 64:
        print_standings(standings, id_to_name, )
        print_pairings(pairs, id_to_name, scores)
    result = run_round(pairs, rnd, p_draw=PROBABILITY_OF_DRAW)
    results += result
    
    for _ in range(NUMBER_OF_DROPS):
        drop_id = np.random.randint(len(players))
        del(players[drop_id])

if NUMBER_OF_PLAYERS <= 64:
    print('----- Final Standings -----')
    print_standings(draw.get_standings(players, results, ROUNDS + [ROUNDS[-1]]), id_to_name, )
    
point_freq = Counter([y for x in [s[a==1].astype(int) for s, a in zip(scores_h, alloc_h)] for y in x])
print('Overall Pairing Score Frequency: ', ', '.join([str(int(k)) + 'pts: ' + str(int(point_freq[k] / 2))
        for k in sorted(point_freq.keys())[::-1]]))
print_byes(pairs_h)

----- Round 1 -----
Goodness of Fit: 15.5
Maximum GOF: 15.5
Frequencies: 16pts: 15, 8pts: 1

 ID        Name                   Wins    Standing    CTB    PML    OML    OCTB    RPLD    Pod    Pod Wins
--------  -------------------  ------  ----------  -----  -----  -----  ------  ------  -----  ----------
a5192112  Ann Blackwell             0           1      0      0      0       0       0      1           0
a3782632  Cynthia Mejia             0           2      0      0      0       0       0      1           0
a4246561  Michael Wilson            0           3      0      0      0       0       0      1           0
a6212529  Mark Woods                0           4      0      0      0       0       0      1           0
a9310410  Tammy Kelly               0           5      0      0      0       0       0      1           0
a7313892  Chelsea Phillips          0           6      0      0      0       0       0      1           0
a4216570  Elizabeth Rich            0           7      0  

----- Round 4 -----
Goodness of Fit: 10.431976947642397
Maximum GOF: 14.5
Frequencies: 16pts: 12, 14pts: 1, 8pts: 1, -22pts: 1, -24pts: 1

 ID        Name                   Wins    Standing       CTB       PML       OML      OCTB    RPLD    Pod    Pod Wins
--------  -------------------  ------  ----------  --------  --------  --------  --------  ------  -----  ----------
a5192112  Ann Blackwell             3           1  1         0         0.444444  0.573099       3      1           0
a8866405  Thomas Combs              3           2  1         0         0.444444  0.362573       3      1           0
a1183234  Matthew Rodriguez         3           3  1         0         0.5       0.54386        2      1           0
a1138215  Amy Benson                2           4  0.719298  0.333333  0.333333  0.666667       3      1           0
a6212529  Mark Woods                2           5  0.719298  0.333333  0.333333  0.573099       3      1           0
a3457313  Rachel Ball               2    

#### Test Code Area

In [10]:
print('\n', tabulate(results, headers=['Player1', 'Player2', 'Winner', 'Round', 'Pod']))
print('\n', tabulate(draw.get_standings(players, results, ROUNDS + [ROUNDS[-1]]), headers=['Player', 'WINS', 'CTB', 'POD', 'PODW', 'PML', 'OML', 'OCTB']))


 Player1    Player2    Winner      Round    Pod
---------  ---------  --------  -------  -----
a5192112   a8470406   a5192112        0      1
a3782632   a5396813   a3782632        0      1
a4246561   a1138215   a1138215        0      1
a6212529   a4051151   a6212529        0      1
a9310410   a9212343   a9212343        0      1
a7313892   a9919847   a7313892        0      1
a4216570   a3422314                   0      1
a1858903   a7799359   a7799359        0      1
a6897440   a8866405   a8866405        0      1
a4466284   a3457313   a3457313        0      1
a3419906   a2697045   a3419906        0      1
a1430571   a9260988   a9260988        0      1
a7074901   a9266155                   0      1
a9986340   a4017057   a9986340        0      1
a8597260   a1302663   a1302663        0      1
a1183234   BYE1       a1183234        0      1
a1183234   a9212343   a1183234        1      1
a5192112   a9986340   a5192112        1      1
a3782632   a1138215   a1138215        1      1
a6212529   

Tournaments are separated into rounds. Every round, each player plays a match against another player, unless they have been awarded a bye. The result of each match (win, loss, or draw), is reported to the Tournament Organizer who enters it into GEM (Game Event Manager) software. During the tournament, players improve their standing by gaining match points as follows:

Win a match (W) = 1 match point
Lose a match (L) = 0 match points
Draw a match (D) = 0 match points (but is significant for tiebreakers)
Bye (B) = 1 match point (A bye is when a player is not assigned an opponent, usually because there is an uneven number of players in the tournament)
Information that is tracked throughout the tournament for each player includes:

Total match points = sum of all Wins and Byes
Cumulative tiebreaker (CTB) = tiebreaker last round / 4 + total match-win points
Player match loss % (PML) = Player total losses / total matches played
Player’s average opponent match loss % (OML%) = sum (PML of each opponent played so far) / number of opponents played
Player’s average opponent CTB (OCTB) = sum (CTB for each opponent played so far) / number of opponents played
Throughout the tournament, player standings (or rank) are determined by the following logic, in this order;

A player with higher total match-win points has a higher standing.
> If multiple players have the same number of match-win points, they tie for that standing, then;
A player with a higher CTB within a tie has the higher standing.
> If multiple players within a tie have the same CTB, then;
A player with a lower PML within a tie has the higher standing.
> If multiple players within a tie have the same PML, then;
A player with a lower OML% within a tie has the higher standing.
> If multiple players within a tie have the same OML%, then;
A player with a higher OCTB within a tie has the higher standing.
> If multiple players within a tie have the same OCTB, then;
Player standing is randomized within the tie.